In [1]:
import numpy as np
import math
import random as rnd
import cv2
from PIL import Image
import matplotlib.pyplot as plt

import glob
%matplotlib inline

9. Два изображения одной и той же сцены сделаны одной и той же   
камерой с такой же матрицей внутренних параметров, как в <code>Задаче 6</code>. Второй   
снимок сделан после поворота камеры на <code>30 градусов</code> вокруг оси X относительно  
начала координат системы отсчета, связанной с камерой (без сдвига). Найти  
матрицу гомографии между двумя изображениями.

In [2]:

k = np.array([[400, 0, 960],
              [0, 400, 540],
              [0, 0, 1]], dtype = np.float64)

rt1 = np.array([[1, 0, 0, 0],
                [0, 1, 0, 0],
                [0, 0, 1, 0]])

rt2 = np.array([[1, 0, 0, 0],
               [0, math.cos(math.radians(30)), -math.sin(math.radians(30)), 0],
              [0, math.sin(math.radians(30)), math.cos(math.radians(30)), 0],], dtype=np.float64)
P1 = k @ rt1
P2 = k @ rt2
P1

array([[400.,   0., 960.,   0.],
       [  0., 400., 540.,   0.],
       [  0.,   0.,   1.,   0.]])

In [3]:
P2

array([[4.00000000e+02, 4.80000000e+02, 8.31384388e+02, 0.00000000e+00],
       [0.00000000e+00, 6.16410162e+02, 2.67653718e+02, 0.00000000e+00],
       [0.00000000e+00, 5.00000000e-01, 8.66025404e-01, 0.00000000e+00]])

In [4]:
def image_generator(P1, P2):
    pairs = []
    for i in range(10):
        x1 = rnd.randint(0, 100)
        y1 = rnd.randint(0, 100)
        z1 = rnd.randint(0, 100)
        pict1 = P1 @ np.array([x1, y1, z1, 1])
        pict1 = pict1 / pict1[2]
        pict2 = P2 @ np.array([x1, y1, z1, 1])
        pict2 = pict2 / pict2[2]
        pairs.append((pict1, pict2))
    return pairs

In [5]:
def homography(pairs):
    x1 = pairs[0][0][0]
    y1 = pairs[0][0][1]
    x2 = pairs[1][0][0]
    y2 = pairs[1][0][1]
    x3 = pairs[2][0][0]
    y3 = pairs[2][0][1]
    x4 = pairs[3][0][0]
    y4 = pairs[3][0][1]
    xp1 = pairs[0][1][0]
    yp1 = pairs[0][1][1]
    xp2 = pairs[1][1][0]
    yp2 = pairs[1][1][1]
    xp3 = pairs[2][1][0]
    yp3 = pairs[2][1][1]
    xp4 = pairs[3][1][0]
    yp4 = pairs[3][1][1]

    matrix=[
	[-x1, -y1, -1, 0, 0, 0, x1*xp1, y1*xp1, xp1],
	[0, 0, 0, -x1, -y1, -1, x1*yp1, y1*yp1, yp1],
	[-x2, -y2, -1, 0, 0, 0, x2*xp2, y2*xp2, xp2],
	[0, 0, 0, -x2, -y2, -1, x2*yp2, y2*yp2, yp2],
	[-x3, -y3, -1, 0, 0, 0, x3*xp3, y3*xp3, xp3],
	[0, 0, 0, -x3, -y3, -1, x3*yp3, y3*yp3, yp3],
	[-x4, -y4, -1, 0, 0, 0, x4*xp4, y4*xp4, xp4],
	[0, 0, 0, -x4, -y4, -1, x4*yp4, y4*yp4, yp4]]
    U, S, Vt = np.linalg.svd(matrix)
    V = Vt.transpose()[:, len(Vt)-1]
    H = V.reshape(3,3)
    return H

Находим матрицу гомографии

In [6]:
pairs = image_generator(P1, P2)
H = homography(pairs)
H

array([[-1.04155540e-03, -1.24986648e-03,  8.08888187e-01],
       [ 1.05997450e-14, -1.60506333e-03,  5.87958025e-01],
       [ 1.21663577e-17, -1.30194425e-06, -1.98963541e-04]])

Более оптимальный способ

In [7]:
H = k @ rt2[:, :3] @ np.linalg.inv(k)
H

array([[ 1.00000000e+00,  1.20000000e+00, -7.76615612e+02],
       [ 0.00000000e+00,  1.54102540e+00, -5.64500000e+02],
       [ 0.00000000e+00,  1.25000000e-03,  1.91025404e-01]])

Проверка: p2 должно быть равно p1

In [8]:
p2 = H @ pairs[0][0]
p2 = p2 / p2[2]
p2

array([1.18459254e+03, 4.98575484e+02, 1.00000000e+00])

In [9]:
p1 = pairs[0][1] / pairs[0][1][2]
p1

array([1.18459254e+03, 4.98575484e+02, 1.00000000e+00])

10. Шахматная доска `8х8` клеток имеет длину клетки `0.2`. Начало мировой  
системы координат находится в левом нижнем углу, оси X и Y направлены вдоль  
сторон клеток, ось Z перпендикулярна плоскости доски. Используя внутренние  
параметры и положение камеры относительно мировой системы координат из  
`задачи 6 (второе домашнее задание)`, и предполагая отсутствие дисторсии, найти  
координаты проекций углов клеток на плоскость изображения. Решить задачу `PnP`,  
используя функцию `cv2.solvePnPGeneric`, для трехмерных координат клеток в  
мировой системе координат и найденных двумерных проекций. Сравнить  
полученные поворот и трансляцию с взятыми из `задачи 6`. Если результаты  
получаются неожиданными, объяснить.

In [27]:
rt = np.array([[math.cos(math.radians(45)), -math.sin(math.radians(45)), 0, 0],
              [math.sin(math.radians(45)), math.cos(math.radians(45)), 0, 0],
              [0, 0, 1, 10]], dtype=np.float64)

k = np.array([[400, 0, 960],
              [0, 400, 540],
              [0, 0, 1]], dtype = np.float64)

P = k @ rt

chess_board = []
for i in range(8):
    for j in range(8):
        chess_board.append(np.array([i * 0.2, j * 0.2, 1, 1], dtype=np.float64))
chess_board = np.array(chess_board, dtype=np.float64)
chess_board = np.around(chess_board, 2)

In [28]:
image_chess_board = []
for item in chess_board:
    point = P @ item
    point /= point[2]
    image_chess_board.append(point)
image_chess_board = np.array(image_chess_board, dtype="double")
image_chess_board = np.around(image_chess_board, 2)

In [29]:
distCoeffs = np.zeros((4,1))

In [30]:
some_p = np.random.choice(len(image_chess_board[:, :2]), 10),
some_p_image_chess_board = image_chess_board[some_p, :2]
some_p_image_chess_board

array([[[929.14, 581.14],
        [934.29, 586.28],
        [939.43, 591.43],
        [934.29, 576.  ],
        [929.14, 570.86],
        [996.  , 576.  ],
        [939.43, 591.43],
        [954.86, 545.14],
        [954.86, 606.85],
        [944.57, 576.  ]]])

In [31]:
some_p_chess_board = chess_board[some_p, :3]
some_p_chess_board

array([[[0.2, 1.4, 1. ],
        [0.4, 1.4, 1. ],
        [0.6, 1.4, 1. ],
        [0.2, 1.2, 1. ],
        [0. , 1.2, 1. ],
        [1.4, 0. , 1. ],
        [0.6, 1.4, 1. ],
        [0. , 0.2, 1. ],
        [1.2, 1.4, 1. ],
        [0.4, 1. , 1. ]]])

In [32]:
assert image_chess_board.shape[0] == chess_board.shape[0]

In [33]:
distCoeffs = np.zeros((4,1))
(_, rotation_vector, translation_vector) = cv2.solvePnP(some_p_chess_board, some_p_image_chess_board, k, distCoeffs, flags=0)

In [34]:
rotation_vector

array([[ 0.0011415 ],
       [-0.00119589],
       [ 0.7854341 ]])

In [35]:
translation_vector

array([[7.49421467e-04],
       [1.42682879e-03],
       [9.99757927e+00]])

11. Найти внутренние параметры камеры и параметры  
дисторсии по изображениям из архива  

https://drive.google.com/file/d/1m6qNqSkZYLZW9YD89zsHIgEgbjWRM-vN/view?usp=sharing.   
Можно использовать сэмпл `example_cpp_calibration` из `OpenCV.`

In [19]:
CHECKERBOARD = (6,9)

criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 30, 0.001)
 
objpoints = []
imgpoints = [] 

objp = np.zeros((1, CHECKERBOARD[0]*CHECKERBOARD[1], 3), np.float32)
objp[0,:,:2] = np.mgrid[0:CHECKERBOARD[0], 0:CHECKERBOARD[1]].T.reshape(-1, 2)
prev_img_shape = None

In [20]:
images = glob.glob('./calibration5/*.jpg')
len(images)

1369

In [21]:
h = 0
w = 0
g = 0
for fname in images:
    img = cv2.imread(fname)
    gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    ret, corners = cv2.findChessboardCorners(gray, CHECKERBOARD, cv2.CALIB_CB_ADAPTIVE_THRESH+
    	cv2.CALIB_CB_FAST_CHECK+cv2.CALIB_CB_NORMALIZE_IMAGE)
    
    if ret == True:
        objpoints.append(objp)
        corners2 = cv2.cornerSubPix(gray,corners,(11,11),(-1,-1),criteria)
        
        imgpoints.append(corners2)
 
        img = cv2.drawChessboardCorners(img, CHECKERBOARD, corners2,ret)
        h,w = img.shape[:2]
        g = gray.shape[::-1]
 
ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(objpoints, imgpoints, g,None,None)
 
print("Camera matrix : \n")
print(mtx)
print("dist : \n")
print(dist)
print("rvecs : \n")
print(rvecs)
print("tvecs : \n")
print(tvecs)

Camera matrix : 

[[413.26373822   0.         480.24163762]
 [  0.         423.30795799 278.90828825]
 [  0.           0.           1.        ]]
dist : 

[[-2.64773952e-01  1.95946347e-01 -1.05666771e-02  1.26042443e-03
   3.04638211e+00]]
rvecs : 

(array([[-0.50169151],
       [-0.28959778],
       [-1.31082311]]), array([[-0.40286608],
       [-0.4815826 ],
       [-1.5596095 ]]), array([[-0.39744002],
       [-0.47981572],
       [-1.56800355]]), array([[-0.39788374],
       [-0.49061085],
       [-1.57230754]]), array([[-0.03149999],
       [ 0.17002026],
       [ 1.48441862]]))
tvecs : 

(array([[-4.47803921],
       [-0.98013657],
       [21.13540045]]), array([[-5.66840734],
       [ 0.04280852],
       [19.77574609]]), array([[-5.64398056e+00],
       [ 1.24099986e-02],
       [ 1.96552162e+01]]), array([[-5.62490842e+00],
       [-1.48474762e-02],
       [ 1.94801982e+01]]), array([[ 1.21399209],
       [-4.63528105],
       [27.15161621]]))


12. Используя матрицу внутренних параметров  
https://drive.google.com/file/d/1A4H84PLy7971Xd1ErS1bRRupWk9_TCYI/view?usp=sharing    
запустить функцию `cv2.undistort()` на изображении   
https://drive.google.com/file/d/1mC0PI9k4q_wJt9iAn6uosEVSJb9PcZIk/view?usp=sharing   
и записать в файл изображение с компенсацией искажений линзы.

In [22]:
k = np.array([[4.2581151397691417e+02, 0, 4.8175802555527360e+02],
              [0, 4.3533255745518812e+02, 2.6743704185634374e+02],
              [0, 0, 1]], dtype = np.float64)

dist_coeffs = np.array([4.4429001834994422e+00, 2.7649030467662392e+00,
    -4.5203901289078106e-03, -1.8217805324766470e-03, 0.,
    4.7056576535214534e+00, 3.9343291290497260e+00, 3.3910745370687528e-01,
    0., 0., 0., 0., 0., 0], dtype=np.float32)

img = cv2.imread('GOPR01170000.jpg')

In [23]:
def imshow(img):
    plt.xticks([])
    plt.yticks([])
    plt.imshow(img)
    pass

In [24]:
img = np.array(img)
xy_undistorted = cv2.undistort(img, k, dist_coeffs)

In [25]:
cv2.imwrite('output.jpg', xy_undistorted)

True